In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
%env PYTHONPATH = /home/adityasidharta/git/kaggle_humpback_new_whale
%env PROJECT_PATH = /home/adityasidharta/git/kaggle_humpback_new_whale
%cd /home/adityasidharta/git/kaggle_humpback_new_whale

env: PYTHONPATH=/home/adityasidharta/git/kaggle_humpback_new_whale
env: PROJECT_PATH=/home/adityasidharta/git/kaggle_humpback_new_whale
/home/adityasidharta/git/kaggle_humpback_new_whale


In [79]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import os
from model.train import fit_model
from sklearn.externals import joblib
from utils.envs import *
from utils.config import *
from model.functions import loss_fn, metric_fn, pred_fn
from utils.common import get_label, remove_new_whale, create_label
from model.arch import se_resnext101
from model.dataset import train_transform, test_transform, TestDataset, TrainDataset
from sklearn.model_selection import train_test_split

In [5]:
train_df = pd.read_csv(train_repo)
train_df = remove_new_whale(train_df)
dev_df, val_df = train_test_split(train_df, test_size = 0.20)

In [6]:
ohe_model, train_image_label, train_ohe_label = create_label(train_df)
joblib.dump(ohe_model, os.path.join(artifacts_path, 'ohe_model.pkl'))

['/home/adityasidharta/git/kaggle_humpback_new_whale/output/artifacts/ohe_model.pkl']

In [7]:
dev_image_label, dev_ohe_label = get_label(dev_df, ohe_model)
val_image_label, val_ohe_label = get_label(val_df, ohe_model)
test_image_label = [x for x in os.listdir(test_path) if x.endswith('.jpg')]

In [8]:
dev_dataset = TrainDataset(dev_image_label, dev_ohe_label, train_path, train_transform, DEVICE)
val_dataset = TrainDataset(val_image_label, val_ohe_label, train_path, test_transform, DEVICE)
test_dataset = TestDataset(test_image_label, test_path, test_transform, DEVICE)

In [9]:
dev_dataloader = DataLoader(dev_dataset, batch_size = 2, shuffle = True, num_workers = 0)
val_dataloader = DataLoader(val_dataset, batch_size = 2, shuffle = False, num_workers = 0)
test_dataloader = DataLoader(test_dataset, batch_size = 2, shuffle = False, num_workers = 0)

In [78]:
N_CLASSES = 5004
model = se_resnext101(n_classes=N_CLASSES, pretrained = True, device = DEVICE).cuda()
n_epoch = 10
dev_dataloader = dev_dataloader
optimizer = optim.Adam(model.parameters(), lr=0.0005)
criterion = nn.BCEWithLogitsLoss()

In [ ]:
fit_model(
    model,
    n_epoch,
    dev_dataloader,
    optimizer,
    criterion,
    loss_fn,
    metric_fn,
    val_dataloader=val_dataloader,
    checkpoint=True,
    model_filename="pytorch",
)